<a href="https://colab.research.google.com/github/MathMachado/eDreams/blob/master/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import main Python libraries



In [0]:
!pip install bamboolib

In [0]:
import pandas as pd
import numpy as np

import matplotlib
import bamboolib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [0]:
url= "https://raw.githubusercontent.com/MathMachado/eDreams/master/Dataframes/train.csv?token=AGDJQ67D2WENLE2YEMFJBPC5SHK26"
df= pd.read_csv(url, sep= ";", index_col='ID', parse_dates = ['DEPARTURE', 'ARRIVAL'])
df.head()

In [0]:
df.info()

In [0]:
# Treating date type columns
from datetime import date, datetime

df2= df.copy()
df2['DEPARTURE_2']= df2['DEPARTURE'] +'/2018'
df2['ARRIVAL_2']= df2['ARRIVAL'] +'/2018'
df2['ARRIVAL_3']= df2['ARRIVAL'] +'/2019'

df2['DEPARTURE_2']= pd.to_datetime(df2['DEPARTURE_2'])
df2['ARRIVAL_2']= pd.to_datetime(df2['ARRIVAL_2'])
df2['ARRIVAL_3']= pd.to_datetime(df2['ARRIVAL_3'])
df2.head()

In [0]:
# Fixing ARRIVAL when ARRIVAL < DEPARTURE
df3= df2.copy()
df3['IS_ARRIVAL_BEFORE_DEPARTURE']= df3['ARRIVAL_2']<df3['DEPARTURE_2']
df3.loc[df3['ARRIVAL_2'] < df3['DEPARTURE_2'], 'ARRIVAL_2']= df3['ARRIVAL_3']

# Calculating the differente between ARRIVAL and DEPARTURE
df3['ARRIVAL_DEPARTURE']= df3['ARRIVAL_2']-df3['DEPARTURE_2']
df3= df3.drop(columns= ['DEPARTURE','ARRIVAL','ARRIVAL_3'], axis= 1)

As we can see above, all rows of the dataframe refers to July.

In [0]:
df3['ARRIVAL_DEPARTURE'].value_counts()

In [0]:
df3.info()

Aparentemente, não há problemas de Missing Values.

In [0]:
df3.head(50)

In [0]:
df3= df3.drop(columns= ['DEPARTURE_2','ARRIVAL_2'], axis= 1)

In [0]:
# Converting column DISTANCE to numeric. For this purpose, I'll cut the distance in the ","

df3[['DISTANCE_2','DISTANCE_REST']] = df3['DISTANCE'].str.split(",",expand=True)
df3['DISTANCE_2']= pd.to_numeric(df3['DISTANCE_2'])
df3[['HAUL_TYPE','DISTANCE','DISTANCE_2','DISTANCE_REST']].head(10)

In [0]:
df3.groupby('HAUL_TYPE').agg({'DISTANCE_2': ['max', 'min', 'median', 'mean']})

Alguma coisa errada com os valores mínimos. Não faz sentido DOMESTIC= 0. Muito menos INTERCONTINENTAL= 0. Vamos investigar um pouco mais isso a seguir. No entanto, vou trabalhar com DISTANCE_2 e desconsiderar DISTANCE_REST.

In [0]:
df4= df3.copy()
df4= df4.drop(columns= ['DISTANCE_REST','DISTANCE', 'IS_ARRIVAL_BEFORE_DEPARTURE'], axis= 1)
df4= df4.rename({'DISTANCE_2': 'DISTANCE'}, axis=1)
df4.head()

In [0]:
# Investigando DISTANCE = 0
df4[df4['DISTANCE']==0].count()

Há 170 registros em que DISTANCE= 0. Eu considero estes registros como Missing Values. Como tratamento, proponho atribuir à estes casos a mediana segmentado por HAUL_TYPE. Ou seja

In [0]:
median_by__HAUL_TYPE= pd.df4.groupby('HAUL_TYPE')['DISTANCE'].median()
median_by__HAUL_TYPE

Vou atribuir o valor 1236 para os Missing values do CONTINENTAL, 632 para os MV do DOMESTIC e 4193 para os MV do INTERCONTINENTAL.

In [0]:
# Identificando os MV. No caso, zeros
df4.loc[df4['DISTANCE'] == 0, 'DISTANCE']= np.nan

# Checking Missing Values
df4.isna().sum()

Vamos tratar os MV da coluna DISTANCE  e DEVICE a seguir:

In [0]:
mask = (df4['HAUL_TYPE'] == 'DOMESTIC') & (df4['DISTANCE'].isna())
df4['DISTANCE'] = median_by__HAUL_TYPE[1].mask(mask, df4['DISTANCE'])

#df4['DISTANCE'][(df4['HAUL_TYPE'] == 'DOMESTIC') & (df4['DISTANCE'].isna())] = median_by__HAUL_TYPE[1]
#median_by__HAUL_TYPE[1]

In [0]:
df4['DISTANCE'].isna()